In [66]:
import requests
import json
import pandas as pd

base_url = 'https://app.ticketmaster.com/discovery/v2/'
api_key = 'apikey=jVksVlDnooATj6zTOXAN0WQ57RyIOEEi'
# &postalCode=23230\

rva = 'countryCode=US\
&dmaId=324\
&radius=5000\
&unit=miles\
&classificationName=music\
&size=100'

attractions = requests.get(f'{base_url}attractions.json?{rva}&{api_key}')
events = requests.get(f'{base_url}events.json?{rva}&{api_key}')
venues = requests.get(f'{base_url}venues.json?{rva}&{api_key}')

print([x.status_code for x in [attractions, events, venues]])
# curl -I url

[200, 200, 200]


In [67]:
attractions_json = json.loads(attractions.content)
events_json = json.loads(events.content)
venues_json = json.loads(venues.content)

In [73]:
events_json

{'_embedded': {'events': [{'name': 'P!NK: Summer Carnival 2023',
    'type': 'event',
    'id': 'vvG1IZ9pNeVNoR',
    'test': False,
    'url': 'https://www.ticketmaster.com/pnk-summer-carnival-2023-inglewood-california-10-05-2023/event/0A005D68C2D2346F',
    'locale': 'en-us',
    'images': [{'ratio': '3_2',
      'url': 'https://s1.ticketm.net/dam/a/4c8/cee4224d-ed9f-4267-81a4-61be2a1f44c8_ARTIST_PAGE_3_2.jpg',
      'width': 305,
      'height': 203,
      'fallback': False},
     {'ratio': '16_9',
      'url': 'https://s1.ticketm.net/dam/a/4c8/cee4224d-ed9f-4267-81a4-61be2a1f44c8_RECOMENDATION_16_9.jpg',
      'width': 100,
      'height': 56,
      'fallback': False},
     {'ratio': '16_9',
      'url': 'https://s1.ticketm.net/dam/a/4c8/cee4224d-ed9f-4267-81a4-61be2a1f44c8_SOURCE',
      'width': 2426,
      'height': 1365,
      'fallback': False},
     {'ratio': '16_9',
      'url': 'https://s1.ticketm.net/dam/a/4c8/cee4224d-ed9f-4267-81a4-61be2a1f44c8_EVENT_DETAIL_PAGE_16_9.jpg

In [134]:
# attractions_df = pd.json_normalize(attractions_json['_embedded']['attractions'])[['id', 'name', 'type']]
events_df = pd.json_normalize(events_json['_embedded']['events'])[['id', 'name', 'type', 'dates.start.localDate', 'dates.start.localTime', 'classifications']]
classifications = []
failed_rows = []

for i in range(0, len(events_df)):
    try:
        a = json.loads(
            str(events_df.classifications[i]).lstrip('[').rstrip(']').replace("'",'"').replace('True', '"True"').replace('False', '"False"')
            )
        classifications.append(a)
    except:
        failed_rows.append(i)
        continue
print(f'failed rows: {len(failed_rows)}')

failed rows: 5


In [132]:
classifications[0]

{'primary': 'True',
 'segment': {'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'},
 'genre': {'id': 'KnvZfZ7vAeA', 'name': 'Rock'},
 'subGenre': {'id': 'KZazBEonSMnZfZ7v6F1', 'name': 'Pop'},
 'type': {'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'},
 'subType': {'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'},
 'family': 'False'}

In [140]:

events_df['primary'] = pd.Series([x['primary'] for x in classifications])
events_df['segment_name'] = pd.Series([x['segment']['name'] for x in classifications])
events_df['segment_id'] = pd.Series([x['segment']['id'] for x in classifications])
events_df = events_df.drop('classifications', axis=1)

In [141]:
events_df[0:10]

,id,name,type,dates.start.localDate,dates.start.localTime,primary,segment_name,segment_id
0,vvG1IZ9pNeVNoR,P!NK: Summer Carnival 2023,event,2023-10-05,18:30:00,True,Music,KZFzniwnSyZfZ7v7nJ
1,Z7r9jZ1AdPM8I,The Commodores w/ The Spinners,event,2023-10-28,20:00:00,True,Music,KZFzniwnSyZfZ7v7nJ
2,vvG10Z4RTQ5UnF,The Spinners,event,2024-03-31,21:00:00,True,Music,KZFzniwnSyZfZ7v7nJ
3,vvG1iZ9N0uj0sH,Coldplay: Music Of The Spheres World Tour - de...,event,2023-10-01,18:30:00,True,Music,KZFzniwnSyZfZ7v7nJ
4,vvG1iZ9NCCusN-,Coldplay: Music Of The Spheres World Tour - de...,event,2023-09-30,18:30:00,True,Music,KZFzniwnSyZfZ7v7nJ
5,vvG1IZ9KBiqNAT,Ed Sheeran: +-=÷x Tour,event,2023-09-23,18:00:00,True,Music,KZFzniwnSyZfZ7v7nJ
6,vvG10Z9iDPSyWH,Koo Koo Kanga Roo,event,2023-09-23,12:00:00,True,Music,KZFzniwnSyZfZ7v7nJ
7,vvG10Z9NnLk4Zd,A Charlie Brown Christmas,event,2023-11-18,14:00:00,True,Music,KZFzniwnSyZfZ7v7nJ
8,vvG10Z9NnLkgZK,A Charlie Brown Christmas,event,2023-11-19,14:00:00,True,Music,KZFzniwnSyZfZ7v7nJ
9,vvG10Z9NnL7VTh,A Charlie Brown Christmas,event,2023-11-17,20:00:00,True,Music,KZFzniwnSyZfZ7v7nJ


In [112]:
events_df

,id,name,type,dates.start.localDate,dates.start.localTime,classifications
0,vvG1IZ9pNeVNoR,P!NK: Summer Carnival 2023,event,2023-10-05,18:30:00,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
1,Z7r9jZ1AdPM8I,The Commodores w/ The Spinners,event,2023-10-28,20:00:00,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
2,vvG10Z4RTQ5UnF,The Spinners,event,2024-03-31,21:00:00,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
3,vvG1iZ9N0uj0sH,Coldplay: Music Of The Spheres World Tour - de...,event,2023-10-01,18:30:00,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
4,vvG1iZ9NCCusN-,Coldplay: Music Of The Spheres World Tour - de...,event,2023-09-30,18:30:00,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
...,...,...,...,...,...,...
95,vvG10Z9YU0Bw1D,Madonna - The Celebration Tour,event,2024-03-04,20:30:00,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
96,vvG10Z9N1nbflx,Coheed and Cambria - NEVERENDER NWFTWM Tour W/...,event,2023-10-07,19:00:00,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
97,G5eYZ9lhsioZu,"Enrique Iglesias, Pitbull, Ricky Martin: The T...",event,2023-11-30,19:00:00,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
98,vvG10Z9nlm1ElX,Laufey - The Bewitched Tour,event,2023-11-17,19:00:00,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."


In [74]:



# venues_df = pd.json_normalize(venues_json['_embedded']['venues'])[['id', 'name', 'type', 'postalCode']]

# venues_df
# pd.json_normalize(pd.json_normalize(events_json['_embedded']['events'])['_embedded.classifications'][0])
events_df

JSONDecodeError: Expecting ',' delimiter: line 1 column 128 (char 127)

In [29]:
classifications = []
for i in range(0, len(events_df)):
    a = json.loads(
        str(events_df.classifications[i]).lstrip('[').rstrip(']').replace("'",'"').replace('True', '"True"').replace('False', '"False"')
        )
    classifications.append(a)

[x['segment']['name'] for x in classifications]

AttributeError: 'DataFrame' object has no attribute 'classifications'

In [ ]:
for i in events_df.iterrows():
    a = str(events_df.classifications[1]).lstrip('[').rstrip(']').replace("'",'"').replace('True', '"True"').replace('False', '"False"')
    a = json.loads(a)
    print(a)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
[d.get('primary') for d in events_df.classifications]

AttributeError: 'list' object has no attribute 'get'

In [ ]:
[x for x in pd.json_normalize(events_json['_embedded']['events'])]

KeyError: 'events'

In [ ]:
pd.data['_embedded']

AttributeError: module 'pandas' has no attribute 'data'